In [1]:
import pandas as pd #
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import math #
from sklearn import preprocessing #
from statistics import mean, stdev

import warnings
warnings.filterwarnings('ignore')

#sklearn
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import auc, precision_score, accuracy_score,recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV #
from sklearn.model_selection import train_test_split #
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import cross_val_score #
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold #
from sklearn.model_selection import cross_validate #

from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import OneHotEncoder #

from sklearn.ensemble import RandomForestClassifier #
import xgboost
from xgboost import XGBClassifier
from scipy import interp

import shap
from shap import TreeExplainer
from category_encoders import *
import pickle
from pickle import load
from time import sleep
from tqdm.notebook import tqdm
import joblib
from joblib import load as jload
from collections import Counter
import seaborn as sns
import time
# some notebook settings
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:90% !important; }</style>"))
np.set_printoptions(precision=4, suppress=True, linewidth=110)

pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 2000)  # or 1000
pd.set_option('display.max_colwidth', None)
sns.set_theme()

# Closer to reality

After discussing, it appears that this inspector team is not inspecting classes 7 (radioactive) and 1(explosives) - which is confirmed by the feature importance of the vanilla model. We thus made the decision to discard those features. Also, it did not make too much sense to use the months as features as it would probably not give that much additional information to inspectors. 

# Functions

In [2]:
def main_fit(X, groupby = None):
    
    import time
    """
    X = dataframe to fit on, containing target column called 'target'
    groupby = existing column name on which to GroupKfold performs crossvalidation. 
    If groupby is given an existing column from X, that column is dummied and cv is performed with StratifiedGroupKfold.
    
    returns: dict() with best precision, mean precision, stdev of precision, recall of THAT model with the best precision and the model object itself
    """ 

    y = X.target.copy()
    X = X.drop('target',axis=1)

    # RF parameter tuning - 
    rf_grid = {
        "max_depth"    : [5,10,15,20] #number of max features to be randomly chosen at each split
      , "n_estimators"  : [100,500,1000,1100] 
      , "max_features" : [len(X.columns),
                          math.trunc(round(math.sqrt(len(X.columns)),0)) - 2, 
                          math.trunc(round(math.sqrt(len(X.columns)),0)) +1 ,
                          math.trunc(round(math.sqrt(len(X.columns)),0))- 1,
                          math.trunc(round(math.sqrt(len(X.columns)),0)),
                          math.trunc(round(math.sqrt(len(X.columns)),0)) +2,
                          round(len(X.columns)/2)
                         ]
    }
    print(rf_grid)
    
    if groupby:
        print('Groupby: ',groupby)
        
        # GroupKFold to preserve the chosen group separately in folds
        skf = StratifiedGroupKFold(n_splits = 10, shuffle=False)
        
        
        final_precision_list = list()
        final_best_models=list()
        final_best_params=list()
        final_recalls = list()
        
        now = time.time()
        #stratified folds with manual fold retain
        for train_index, test_index in skf.split(X, y, groups=list(X[groupby])):
            
            precision_list = list()
            best_params = list()
            recall_list=list()
            best_models=list()
            
            #dummy the chosen column and add it to the df while deleting the original one
            one_hot = pd.get_dummies(X[groupby], prefix=groupby)
            X_df = pd.concat([X, one_hot],axis=1).drop([groupby], axis=1)
            
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X_df.iloc[train_index], X_df.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            print(groupby,' used in training :', X.iloc[train_index][groupby].unique())
            print(groupby,' used in testing :', X.iloc[test_index][groupby].unique())

            #fit the grid manually
            now = time.time()
            
            #all possible combinations of hyperparams (basically the inner cv)
            for max_depth in rf_grid['max_depth']:
                for n_estimators in rf_grid['n_estimators']:
                    for max_features in rf_grid['max_features']:
                        
                        #fit
                        clf = RandomForestClassifier(max_depth = max_depth,n_estimators = n_estimators, max_features = max_features)
                        clf.fit(X_train,y_train)
                        
                        #predict
                        y_predict = clf.predict(X_test)
                        
                        #retain performance metrics
                        precision_list.append(precision_score(y_test,y_predict))
                        best_models.append(clf)
                        recall_list.append(recall_score(y_test,y_predict))
                        
                    
            
            #what are the best hyperparams for this particular groupskfold split
            
            best_precision = max(precision_list)
            best_recall = recall_list[precision_list.index(best_precision)]
            
            best_model = best_models[precision_list.index(best_precision)]
            best_param = {'n_estimators': best_model.n_estimators,
                         'max_depth': best_model.max_depth,
                         'max_features': best_model.max_features}
            
            print('best precision of the run: ',best_precision)
            print('best model of the run: ',best_model)
            print('recall: ',best_recall)
            
            #create the final lists for all folds
            final_precision_list.append(best_precision)
            final_best_models.append(best_model)
            final_best_params.append(best_param)
            final_recalls.append(best_param)
            
            final_precision = max(final_precision_list)
            final_precision_index = final_precision_list.index(final_precision)
            
            
        print('Time it took to fit: ',time.time()-now)
        #attribute the best params to the dict
        print('This is the best precision score of your model: ', final_precision)
        print('This is the mean precision score: ',round(mean(final_precision_list),2),' with a std of: ', round(stdev(final_precision_list),2))
        print('And for that model, recall is :', final_recalls[final_precision_index])
        print('Your best params are :', final_best_params[final_precision_index])
        
        results = dict()
        results = {'best_precision': final_precision,
                  'mean_precision': round(mean(final_precision_list),2),
                  'std_precision': round(stdev(final_precision_list),2),
                  'recall': final_recalls[final_precision_index],
                  'best_model':best_model}
        
    else:
    
         # simple, stratified cross validation object (no groupby)
        outer_cv = StratifiedKFold(n_splits=10, shuffle=False)
    
        final_precision_list = list()
        final_best_models=list()
        final_best_params=list()
        final_recalls = list()
        
        now = time.time()
        
        #stratified folds with manual fold retain
        for train_index, test_index in outer_cv.split(X, y):
            
            precision_list = list()
            best_params = list()
            recall_list=list()
            best_models=list()
            
            
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            #print(' Countries used in training :', X.iloc[train_index]['country'].unique())
            #print(' Countries used in testing :', X.iloc[test_index]['country'].unique())
            
            #all possible combinations of hyperparams 
            for max_depth in rf_grid['max_depth']:
                for n_estimators in rf_grid['n_estimators']:
                    for max_features in rf_grid['max_features']:
                        
                        #fit
                        clf = RandomForestClassifier(max_depth = max_depth,n_estimators = n_estimators, max_features = max_features)
                        clf.fit(X_train,y_train)
                        
                        #predict
                        y_predict = clf.predict(X_test)
                        
                        #retain performance metrics
                        precision_list.append(precision_score(y_test,y_predict))
                        best_models.append(clf)
                        recall_list.append(recall_score(y_test,y_predict))
                        
                    
            
            #what are the best hyperparams for this particular simple split
            
            best_precision = max(precision_list)
            best_recall = recall_list[precision_list.index(best_precision)]
            best_model = best_models[precision_list.index(best_precision)]
            
            best_param = {'n_estimators': best_model.n_estimators,
                         'max_depth': best_model.max_depth,
                         'max_features': best_model.max_features}
            
            
            #create the final lists for all folds
            final_precision_list.append(best_precision)
            final_best_models.append(best_model)
            final_best_params.append(best_param)
            final_recalls.append(best_recall)
            
            final_precision = max(final_precision_list)
            final_precision_index = final_precision_list.index(final_precision)
            
            
        print('Time it took to fit: ',time.time()-now)
        #attribute the best params to the dict
        print('This is the best precision score of your model: ', final_precision)
        print('This is the mean precision score: ',round(mean(final_precision_list),2),' with a std of: ', round(stdev(final_precision_list),2))
        print('And for that model, recall is :', final_recalls[final_precision_index])
        print('Your best params are :', final_best_params[final_precision_index])
        
        results = dict()
        results = {'best_precision': final_precision,
                  'mean_precision': round(mean(final_precision_list),2),
                  'std_precision': round(stdev(final_precision_list),2),
                  'recall': final_recalls[final_precision_index],
                  'best_model':best_model}

    return results


In [3]:
#def a function for this shap_tree comparison
def importance_comparison(X,best_model):
    """
    X: dataframe with data used to fit
    shap_values: dataframe with shap values 
    """
    #generate the explanations
    
    print('Calculating SHAP values, this might take a little...')
    explainer = shap.TreeExplainer(model=best_model)
    shap_values = explainer.shap_values(X.values)
    
    #ranking features based on mean absolute shap. Indexing like this since it is a RF with two sets of SHAP values.
    shap_df = pd.DataFrame(shap_values[1],columns = X.columns)
    shap_mean_s = abs(shap_df).mean(axis=0)
    shap_mean_df = pd.DataFrame(shap_mean_s,columns = ['shap_importance'])
    shap_mean_df = shap_mean_df.sort_values(by='shap_importance',ascending=False)
    shap_mean_df.reset_index(inplace=True)
    shap_mean_df.reset_index(inplace=True)
    shap_mean_df.set_index('index',inplace=True)
    shap_mean_df.columns=['shap_rank','shap_importance']
    
    #ranking features based on value importance
    print(len(X.columns),best_model.feature_importances_)
    value_imp = pd.DataFrame({"feature": X.columns, "tree_importance": best_model.feature_importances_})
    value_imp = value_imp.sort_values(['tree_importance'], ascending=False)
    value_imp.set_index('feature',inplace=True)
    value_imp['tree_rank'] = list(range(0,len(value_imp)))
    #joining them
    df = value_imp.join(shap_mean_df)
    
    return df[['tree_rank','shap_rank']]

# Experiments

In [4]:
df = pd.read_csv('data/clean_holmes_2019.csv')
df.head()

,class_1_1_explosives_mass,class_1_2_explosives_projection,class_1_3_explosives_fire,class_1_4_explosives_low_risk,class_1_5_explosives_mass_very_insensitive,class_1_6_explosives_very_insensitive,class_2_1_flammable_gases,class_2_2_non_flammable_non_toxic_gases,class_2_3_toxic_gases,class_3_flammable_liquids,class_4_1_flammable_solids,class_4_2_spontaneous_combustion,class_4_3_flammable_gases_in_contact_water,class_5_1_oxidizing_substances,class_5_2_organic_peroxides,class_6_1_toxic_substances,class_6_2_infectious_substances,class_7_radioactive_material,class_8_corrosives,class_9_miscellaneous_dangerous_goods,PG_I_high_danger,PG_II_medium_danger,PG_III_low_danger,Quantity_total,Nr_dif_UNNRs,Packaging_DRUM_yn,Packaging_JERRYCAN_yn,Packaging_BOX_yn,Packaging_BAG_yn,Packaging_COMPOSITE_yn,Packaging_UNPACKED_yn,Packaging_IBC_yn,Packaging_LP_yn,Packaging_Bulk_container_yn,Packaging_PR_yn,METAL_packaging_yn,WOODEN_packaging_yn,PLASTIC_packaging_yn,OTHER_packing_material_yn,Prisma_object_type_corrected_Container,Prisma_object_type_corrected_IMO or UN transport tank,Prisma_in_outgoing_corrected_Incoming,Prisma_in_outgoing_corrected_Outgoing or Transshipment,Prisma_terminal_corrected_APM Terminals Maasvlakte II,Prisma_terminal_corrected_APM Terminals Rotterdam,Prisma_terminal_corrected_ECT Delta Terminal,Prisma_terminal_corrected_Euromax Terminal,Prisma_terminal_corrected_Other terminal,Prisma_terminal_corrected_Rotterdam World Gate Terminal (RWG),Prisma_region_origin_incoming_corrected_Africa,Prisma_region_origin_incoming_corrected_China,Prisma_region_origin_incoming_corrected_India,Prisma_region_origin_incoming_corrected_Latin_America,Prisma_region_origin_incoming_corrected_Middle_East,Prisma_region_origin_incoming_corrected_Rest_of_the_world,Prisma_region_origin_incoming_corrected_South_East_Asia,Prisma_region_destination_incoming_corrected_Europe,Prisma_region_destination_incoming_corrected_Netherlands,Prisma_region_destination_incoming_corrected_Rest_of_the_world,Prisma_cargo_agent_corrected_8 - CMA CGM (Holland) B.V.,Prisma_cargo_agent_corrected_20 - Hapag-Lloyd Rotterdam,Prisma_cargo_agent_corrected_22 - Hyundai Merchant Marine,Prisma_cargo_agent_corrected_24 - Maersk Line,Prisma_cargo_agent_corrected_25 - Mediterranean Shipping Company,Prisma_cargo_agent_corrected_27 - Mueller Liner Agency,Prisma_cargo_agent_corrected_30 - Ocean Network Express (ONE),Prisma_cargo_agent_corrected_Other_cargo_agent,target,spec_container_type_Bulk Container,spec_container_type_General Purpose Container,spec_container_type_IMO Tankcontainer,spec_container_type_Reefer Container,spec_container_type_UN MEGC,spec_container_type_UN transporttank,transportation_mode_Containership,transportation_mode_General Cargo ship,transportation_mode_RoRo,transportation_mode_Roro,transportation_mode_missing,transportation_type_Deep Sea,transportation_type_Short Sea,transportation_type_missing,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,7250.0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,10050.0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,12300.0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,16000.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0

In [6]:
cols_to_del = list()
for col in df.columns.tolist():
    if 'class_1_' in col:
            cols_to_del.append(col)
        
    if 'month' in col:
        cols_to_del.append(col)
        
cols_to_del.append('class_7_radioactive_material')

In [ ]:
# Save model with the best params
#pickle.dump(best_model, open("best_model_real_GS.pickle","wb"))

Experiments: Check performance difference between runs with and without StratifiedGroupKfold:
1. Replace regions with countries
2. Replace countries with GDP.
3. Check influence of Cargo agents

For each, compare performance metrics and shifts in feature importance (SHAP vs tree feature importance). 

## Checking best model score with initial columns data, no groupKfold
Meaning only regions, no countries, no months, cargo agents grouped. 

In [ ]:
main_results = main_fit(df,groupby=None)

{'max_depth': [5, 10, 15, 20], 'n_estimators': [100, 500, 1000, 1100], 'max_features': [74, 7, 10, 8, 9, 11, 37]}


## Experiment 1: adding countries, removing regions

In [ ]:
df = pd.read_csv('clean_holmes_2019.csv')

### Selecting relevant columns

In [ ]:
#just clean it and get origin countries and cargo agent
#700 tons is too much for one container, therefore it is being discarded
countries_exp1 = pd.read_csv('HOLMES_data_2019_all.csv')
countries_exp1 = countries_exp1[countries_exp1['Quantity_total']<100000]
countries_exp1 = countries_exp1[countries_exp1['Quantity_total']>10]
countries_exp1 = countries_exp1['ORIGINAL_Prisma_incoming_country_origin']
countries_exp1.reset_index(inplace=True,drop=True)

#replacing the region with the actual countries
col_to_del = list()
for col in df.columns:
    if 'Prisma_region_origin_incoming' in col:
        col_to_del.append(col)


df.drop(col_to_del,axis=1,inplace=True)
df['country'] = countries_exp1
#df.country

### Looking at scores with countries, no GroupKfold

In [ ]:
one_hot = pd.get_dummies(df['country'],prefix='country')
df = df.join(one_hot)
df = df.drop('country',axis=1)

In [ ]:
results_countries_codes = main_fit(df,groupby=None)

### SHAP feature importance and feature importance comparison 

In [ ]:
explainer = shap.TreeExplainer(model=results_countries_codes.best_model)
shap_values = explainer.shap_values(df.values)

In [ ]:
#generate the explanation object
explanation_object = shap.Explanation(
values= shap_values,
base_values=explainer.expected_value[1],
data=X.values,
feature_names=X.columns)

In [ ]:
#SHAP
display = 25 
shap.summary_plot(explanation_object.values[1], max_display=display, plot_type='bar', feature_names=df.columns)

### Looking at scores with countries as GroupKfold 

In [ ]:
#just clean it and get origin countries and cargo agent
#700 tons is too much for one container, therefore it is being discarded
countries_exp1 = pd.read_csv('HOLMES_data_2019_all.csv')
countries_exp1 = countries_exp1[countries_exp1['Quantity_total']<100000]
countries_exp1 = countries_exp1[countries_exp1['Quantity_total']>10]
countries_exp1 = countries_exp1['ORIGINAL_Prisma_incoming_country_origin']
countries_exp1.reset_index(inplace=True,drop=True)

#replacing the region with the actual countries
col_to_del = list()
for col in df.columns:
    if 'Prisma_region_origin_incoming' in col:
        col_to_del.append(col)
        
    if 'month' in col:
        col_to_del.append(col)

df.drop(col_to_del,axis=1,inplace=True)
df['country'] = exp1_df
#df.country


In [ ]:
#scroll all the way down to see the results
results_countries_codes_GroupKfold = main_fit(df,groupby='country')

In [ ]:
####Conclusion: If country is missing, then performance drops 16% on a test set with lots of missing countries. 
####

In [ ]:
len(df.country.dropna())

## Experiment 2: GDP instead of countries

In [ ]:
#reading the dictionary
filename = 'gdp_dict.pickle'
#open the file to which it will be read from
fileobj = open(filename, 'rb')
#dump the object
gdp_dict = pickle.load(fileobj)
#close
fileobj.close()

In [ ]:
gdp_exp = pd.read_csv('data/HOLMES_data_2019_all.csv')

gdp_exp = gdp_exp[gdp_exp['Quantity_total']<100000]
gdp_exp = gdp_exp[gdp_exp['Quantity_total']>10]

gdp_exp.reset_index(inplace=True,drop=True)
#getting the country
countries = gdp_exp['ORIGINAL_Prisma_incoming_country_origin']

#deleting the already-dummied columns and the month column which we won't use
col_to_del = list()
for col in df.columns:
    if 'Prisma_region_origin_incoming' in col:
        col_to_del.append(col)
    if 'month' in col:
        col_to_del.append(col)
        
#the final df o which to run
df_gdp = df.drop(col_to_del,axis=1)
#df.drop('Unnamed: 0',axis=1,inplace=True)

#to run without any groupKfold, all categorical variables used for experiments must be 
df_gdp['country'] = countries

#one_hot = pd.get_dummies(df_cargo['country'], prefix='country')
#df_gdp = pd.concat([df_cargo, one_hot],axis=1).drop(['country'], axis=1)

df_gdp['gdp'] = list(map(gdp_dict.get,df_gdp['country']))
df_gdp[['country','gdp']]
df_gdp.drop('country',axis=1,inplace=True)

#replacing the None with Nans
df_gdp[df_gdp['gdp'].isnull()]=np.nan

In [ ]:
results_gdp=main_fit(df_gdp)

### SHAP feature importance and feature importance comparison 

In [ ]:
explainer = shap.TreeExplainer(model=results_gdp['best_model'])
shap_values = explainer.shap_values(df_gdp.values)

In [ ]:
#generate the explanation object
def define_xai_obj(data, index):
    
    explanation_object = shap.Explanation(
        values= shap_values,
        base_values=explainer.expected_value[index],
        data=data.values,
        feature_names=data.columns)

    return explanation_object

In [ ]:
explanation_object = define_xai_obj(df_gdp,1)

In [ ]:
#SHAP
display = 25 
shap.summary_plot(explanation_object.values[1], max_display=display, plot_type='bar', feature_names=df.columns)

In [ ]:
importance_comparison(df_gdp,gdp_results['best_model'])

In [ ]:
explainer = shap.TreeExplainer(model=gdp_results['best_model'])
shap_values = explainer.shap_values(df_gdp.values)

In [ ]:
shap_values[1]

In [ ]:
df_gdp.columns

In [ ]:
#ranking features based on mean absolute shap. Indexing like this since it is a RF with two sets of SHAP values.
shap_df = pd.DataFrame(shap_values[1],columns = df_gdp.columns)
shap_mean_s = abs(shap_df).mean(axis=0)
shap_mean_df = pd.DataFrame(shap_mean_s,columns = ['shap_importance'])
shap_mean_df = shap_mean_df.sort_values(by='shap_importance',ascending=False)
shap_mean_df.reset_index(inplace=True)
shap_mean_df.reset_index(inplace=True)
shap_mean_df.set_index('index',inplace=True)
shap_mean_df.columns=['shap_rank','shap_importance']

In [ ]:
len(gdp_results['best_model'].feature_importances_)

In [ ]:


#ranking features based on value importance
value_imp = pd.DataFrame({"feature": df_gdp.columns, "tree_importance": gdp_results['best_model'].feature_importances_})
value_imp = value_imp.sort_values(['tree_importance'], ascending=False)
value_imp.set_index('feature',inplace=True)
value_imp['tree_rank'] = list(range(0,len(value_imp)))
#joining them
df = value_imp.join(shap_mean_df)

df[['tree_rank','shap_rank']]

## Experiment 3: Cargo agents with and without GroupKFold (with countries)

In [ ]:
cargo_exp = pd.read_csv('HOLMES_data_2019_all.csv')
#cargo_exp.ORIGINAL_Prisma_cargo_agent.unique()

#delete first character of cargo agent names if its a space
cargo_exp['ORIGINAL_Prisma_cargo_agent'] = cargo_exp.ORIGINAL_Prisma_cargo_agent.str.lstrip(' ')
#cargo_exp.ORIGINAL_Prisma_cargo_agent.unique()

In [ ]:
#just clean it and get origin countries and cargo agent
#700 tons is too much for one container, therefore it is being discarded
cargo_exp = cargo_exp[cargo_exp['Quantity_total']<100000]
cargo_exp = cargo_exp[cargo_exp['Quantity_total']>10]

#getting the country
countries = cargo_exp['ORIGINAL_Prisma_incoming_country_origin']

#extracting just the series with cargo agents
cargo_exp = cargo_exp['ORIGINAL_Prisma_cargo_agent']
cargo_exp.reset_index(inplace=True,drop=True)

#deleting the already-dummied columns
col_to_del = list()
for col in df.columns:
    if 'Prisma_region_origin_incoming' in col:
        col_to_del.append(col)

    if 'Prisma_cargo_agent' in col:
        col_to_del.append(col)
        
df_cargo = df.drop(col_to_del,axis=1)
#df.drop('Unnamed: 0',axis=1,inplace=True)

#to run without any groupKfold, all categorical variables used for experiments must be 
df_cargo['country'] = countries
df_cargo['cargo_agent'] = cargo_exp
one_hot = pd.get_dummies(df_cargo['country'], prefix='country')
df_cargo = pd.concat([df_cargo, one_hot],axis=1).drop(['country'], axis=1)

### without GroupKfold

In [ ]:
one_hot = pd.get_dummies(df_cargo['cargo_agent'], prefix='cargo_agent')
df_cargo = pd.concat([df_cargo, one_hot],axis=1).drop(['cargo_agent'], axis=1)
results_cargo_agent = main_fit(df_cargo)

### with GroupKfold

In [ ]:
#this time we only dummy the countries, the cargo agent is dummied in the main_fit function for the GroupKfold.
df_cargo = df.drop(col_to_del,axis=1)
df_cargo['country'] = countries
df_cargo['cargo_agent'] = cargo_exp

one_hot = pd.get_dummies(df_cargo['country'], prefix='country')
df_cargo = pd.concat([df_cargo, one_hot],axis=1).drop(['country'], axis=1)

results_cargo_agent_groupKFold = main_fit(df_cargo,groupby='cargo_agent')

In [ ]:
#this time, if the testing contains missing Cargo agent, the precision is one of the best. 

## Conclusion: Experiment Results

In [ ]:
index = ['vanilla','countries','countries_group','cargo','cargo_group','gdp','gdp_group']

In [ ]:
cargo_group = pd.DataFrame.from_dict(results_cargo_agent_groupKFold).iloc[0]

In [ ]:
cargo = pd.DataFrame.from_dict(results_cargo_agent).iloc[0]

In [ ]:
country

In [ ]:
country_group

In [ ]:
main_results

In [ ]:
results_df=pd.DataFrame(index=index,columns=main_results.keys())
results_df

In [ ]:
results_df.loc['vanilla',:] = main_results
results_df.loc['countries',:] = country
results_df.loc['countries_group',:] = country_group
results_df.loc['cargo',:] = cargo
results_df.loc['cargo_group',:] = cargo_group
results_df.loc['gdp']=gdp_results
results_df.drop('gdp_group',axis=0,inplace=True)
results_df.to_csv('results_experiments.csv')

In [ ]:
results_df.drop('best_model',axis=1)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
country = pd.Series()
country['best_precision'] = 0.781
country['mean_precision'] = 0.77
country['std_precision'] = 0.01
country['recall'] = 0.957
country['best_model'] = DecisionTreeClassifier(max_depth=20,max_features=64)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
country_group = pd.Series()
country_group['best_precision'] = 0.86
country_group['mean_precision'] = 0.78
country_group['std_precision'] = 0.07
country_group['recall'] = 0.962
country_group['best_model'] = DecisionTreeClassifier(max_depth=20,max_features=58)

In [ ]:
results_df.to_csv('experiment_results03152022.csv')

In [ ]:
##finding the precision on the OOS
print('Precision on OOS: ',precision_score(y_OOS,y_pred_OOS))
print('Accurracy on OOS: ',accuracy_score(y_OOS,y_pred_OOS))
print('Recall on OOS: ',recall_score(y_OOS,y_pred_OOS))
print('ROC_AUC_score OOS: ',roc_auc_score(y_OOS,y_pred_OOS))

## Dashboard Train Performance

In [ ]:
#just like before, we do the performance check for the dashboard etc.
# (not-nested, but simple) cross validation object
inner_cv = StratifiedKFold(n_splits=10, shuffle=True) 

# RF parameter tuning - 
rf_grid = {
    "max_depth"    : [5,10,15,20] #number of max features to be randomly chosen at each split
  , "n_estimators"  : [100,500,1000,1100] 
  , "max_features" : [len(X.columns),
                      math.trunc(round(math.sqrt(len(X.columns)),0)) - 2, 
                      math.trunc(round(math.sqrt(len(X.columns)),0)) +1 ,
                      math.trunc(round(math.sqrt(len(X.columns)),0))- 1,
                      math.trunc(round(math.sqrt(len(X.columns)),0)),
                      math.trunc(round(math.sqrt(len(X.columns)),0)) +2,
                      round(len(X.columns)/2)
                     ]
}
print(rf_grid)

# RF grid search (combines all possible parameters and gives the parameters for the best cv score)
rf_estimator = GridSearchCV(
    estimator = RandomForestClassifier()
    , param_grid = rf_grid
    , scoring = 'precision'
    , cv = inner_cv
    , n_jobs=-1)



In [ ]:
#takes about 40-60 minutes
#we add the label, predicted score and fold to lists and then save it to a csv.
rs = 42
labels = list()
predictions = list()
fold = list()
i = 0

#stratified folds with manual fold retain
skf = StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
for train_index, test_index in skf.split(X, y):
     
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #fit the grid and retain the best model
    rf_estimator.fit(X_train,y_train)
    best_model = rf_estimator.best_estimator_
    
    #use the best model to get only predicted class 1 scores
    probs = rf_estimator.predict_proba(X_test)[:,1]
    
    #retain ground truth labels,predictions and fold numbers
    labels.append(y_test.tolist())
    predictions.append(probs)
    fold.append([i]*len(probs))
    
    #show progress
    print(i)
    i=i+1

In [ ]:
#flattening
labels_flat   = [val for sublist in labels for val in sublist]
predictions_flat   = [val for sublist in predictions for val in sublist]
fold_flat   = [val for sublist in fold for val in sublist]

In [ ]:
dev_dict = {'labels':labels_flat,
           'predictions':predictions_flat,
           'fold':fold_flat}
dev_set = pd.DataFrame.from_dict(dev_dict)


In [ ]:
dev_set.to_csv('dev_set_GS_zeevaart_real.csv',index=False)
#to be used in the dashboard to generate plots and stuff

### Notebook-specific model performance

In [ ]:
#print(date.datetime.now())
# Nested CV with parameter optimization
outer_cv = StratifiedKFold(n_splits=10, shuffle=True)
scoring = ('precision', 'recall', 'average_precision', 'roc_auc')
nested_score = cross_validate(rf_estimator, X=X, y=y, scoring = scoring ,cv=outer_cv)

In [ ]:
#just printing some performance metrics
mean_test_roc_auc = nested_score['test_roc_auc'].mean()
mean_test_roc_auc 

mean_test_precision = nested_score['test_precision'].mean()
mean_test_precision

mean_test_recall = nested_score['test_recall'].mean()
mean_test_recall

mean_test_average_precision = nested_score['test_average_precision'].mean()
mean_test_average_precision